In [ ]:
# !pip install ipywidgets
# !pip install datasets
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="./cats_vs_dogs")
dataset = dataset["train"]
dataset

In [ ]:
dataset = dataset.shuffle(seed=1234).select(range(100)) # 🤔
dataset = dataset.train_test_split(test_size=0.3, seed=1234) # 🤔
dataset

In [ ]:
dataset["train"][0]["image"]

In [ ]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label

In [ ]:
# !pip install torch torchvision torchaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# !pip install transformers
from transformers import AutoImageProcessor

checkpoint = "microsoft/resnet-50" # 🤔
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
image_processor

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor # 🤔

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = image_processor.size["shortest_edge"] if "shortest_edge" in image_processor.size else (image_processor.size["height"], image_processor.size["width"])
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize]) # 🤔

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

dataset = dataset.with_transform(transforms)
dataset

In [ ]:
dataset["train"][0]["pixel_values"]

In [ ]:
# !pip install ray[tune]

from ray import tune 

def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([1, 2, 3]),
    }

In [ ]:
# !pip install accelerate
# !pip install evaluate
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer, DefaultDataCollator
import evaluate
import numpy as np

# model = AutoModelForImageClassification.from_pretrained(
#     checkpoint,
#     num_labels=len(labels),
#     id2label=id2label,
#     label2id=label2id,
#     ignore_mismatched_sizes=True,
# )

def model_init(trial):
    return AutoModelForImageClassification.from_pretrained(
        checkpoint,
        num_labels=len(labels),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True,
    )

training_args = TrainingArguments(
    output_dir="my_image_model", # 🤔
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch", # 🤔
    learning_rate=1e-5, # 🤔
    per_device_train_batch_size=1, # 🤔
    gradient_accumulation_steps=1, # 🤔
    per_device_eval_batch_size=1, # 🤔
    num_train_epochs=1, # 🤔
    warmup_ratio=0.1, # 🤔
    logging_steps=10, # 🤔
    load_best_model_at_end=True, # 🤔
    metric_for_best_model="accuracy",
)

accuracy = evaluate.load("accuracy")
data_collator = DefaultDataCollator()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=None,
    model_init=model_init,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

# trainer.train()
# trainer.save_model("my_final_image_model") # 🤔

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    hp_space=ray_hp_space,
    n_trials=10,
)

# Computer\HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\FileSystem
# LongPathsEnabled -> 1

In [ ]:
best_trial.hyperparameters

In [ ]:
dataset = load_dataset("imagefolder", data_dir="./cats_vs_dogs")
dataset

In [ ]:
image = dataset["train"]["image"][1] # 🤔
image

In [ ]:
from transformers import pipeline

classifier = pipeline("image-classification", model="./my_final_image_model")
classifier(image)